*****************************************************************
#  The Social Web: data representation
- Instructors: Jacco van Ossenbruggen.
- TAs: Ayesha Noorain, Alex Boyko, Caio Silva, Elena Beretta, Mirthe Dankloff.
- Exercises for Hands-on session 2
*****************************************************************

In this session you are going to mine data in various microformats. You will see the differences in what each of the formats can contain and what purpose they serve. We will start by looking at geographical data.

Prerequisites:
- Python 3.8
- Python packages: requests, BeautifulSoup4, HTMLParser, rdflib


In [3]:
# If you're using a virtualenv, make sure it's activated before running
# this cell!
!pip install requests
!pip install BeautifulSoup4
!pip install HTMLParser
!pip install rdflib

# IMPORTANT!!!
# IN ORDER TO AVOID ERROR: "ERROR: Could not build wheels for cryptography which use PEP 517 and cannot be installed directly" when installing the requests package
# CHECK THE requirements.txt file afterwards install them via the following command: pip install -r requirements.txt

     -------------------------------------- 128.2/128.2 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for HTMLParser: started
  Running setup.py install for HTMLParser: finished with status 'done'


  DEPRECATION: HTMLParser is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


     ------------------------------------- 500.3/500.3 kB 10.4 MB/s eta 0:00:00
     ---------------------------------------- 41.7/41.7 kB 2.0 MB/s eta 0:00:00


##  Exercise 1

Even if web pages do not use microformat, interesting data can often be extracted from the HTML. You may use packages such as BeautifulSoup to extract arbitrary pieces of data from any HTML page.
The example below shows how we can find the URL of first image in the infobox table of the wikipedia page on Amsterdam. Tip: compare the code below with HTML source code of the wikipedia page: the image url is in the "src" attribute of the "img" element of in the "table" element with class="infobox".

In [4]:
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup

# This script requires you to add a url of a page with geotags to the commandline, e.g.
# python geo.py 'http://en.wikipedia.org/wiki/Amsterdam'
URL = 'https://en.wikipedia.org/wiki/Amsterdam'

req = requests.get(URL, headers={'User-Agent' : "Social Web Course Student"})
soup = BeautifulSoup(req.text)
# print(req.text)
image1 = soup.findAll('table', class_='infobox')[0].find('img')
print(image1['src'])  


//upload.wikimedia.org/wikipedia/commons/thumb/b/be/KeizersgrachtReguliersgrachtAmsterdam.jpg/270px-KeizersgrachtReguliersgrachtAmsterdam.jpg


Extracting coordinates from a webpage and reformatting them in the geo microformat (based on Example 8-1 in Mining the Social Web). Note that wikipages may encode long/lat information in different ways. On of the ways used by the Amsterdam wikipedia page is in a span element that is not shown to the user: 
<span class="geo">52.367; 4.900</span>
This span element has a single child: len(geoTag == 1) and no further structure, we have to manually get the long/lat by splitting the string on the ';' semicolon.

In [5]:

geoTag = soup.find(True, 'geo')
print(geoTag)

if geoTag and len(geoTag) > 1:
        lat = geoTag.find(True, 'latitude').string
        lon = geoTag.find(True, 'longitude').string
        print ('Location is at'), lat, lon
elif geoTag and len(geoTag) == 1:
        (lat, lon) = geoTag.string.split(';')
        (lat, lon) = (lat.strip(), lon.strip())
        print (('Location is at'), lat, lon)
else:
        print ('Location not found')


<span class="geo">52.367; 4.900</span>
Location is at 52.367 4.900


### Task 1

Can you convert the output of Exercise 1 into KML? Here is the KML documentation: https://developers.google.com/kml/documentation/?csw=1 and here you can find a simple example of how it is used: https://renenyffenegger.ch/notes/tools/Google-Earth/kml/index

Visualise the point in Google Maps using the following code example: https://developers.google.com/maps/documentation/javascript/examples/layer-kml-features
You will have to create your own KML file for the custom map layer, and provide a URL to the KML file inside the JavaScript code, which means that you have to upload the file somewhere. You can use a service like http://pastebin.com/ to obtain a URL for your KML file —> paste the code there and request the RAW format URL; use this one in this Task1.

Is KML a microformat, why (not)?

In [ ]:
#KML is a microformat, moreover is an XML-based format provided by Google. 
#It is a microformat because it serves as consistent and descriptive metadata about an element, 
#it is representing a certain type of data, in our case, geographic coordinates.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Placemark>
    <name>VrijeUniversity</name>
    <description>This is the location of Vrije University Amsterdam.</description>
    <Point>
      <coordinates>52.333730555888685, 4.8656233385210035</coordinates>
    </Point>
  </Placemark>
</kml>

In [6]:
function initMap() {
  const map = new google.maps.Map(document.getElementById("map"), {
    zoom: 12,
    center: { lat: 52.33, lng: 4.86 },
  });
  const kmlLayer = new google.maps.KmlLayer({
    url: "https://raw.githubusercontent.com/bobobo199733/the-social-web/main/vrije_location.kml",
    suppressInfoWindows: true,
    map: map,
  });

  kmlLayer.addListener("click", (kmlEvent) => {
    const text = kmlEvent.featureData.description;

    showInContentWindow(text);
  });

  function showInContentWindow(text) {
    const sidebar = document.getElementById("sidebar");

    sidebar.innerHTML = text;
  }
}
window.initMap = initMap;


SyntaxError: invalid syntax. Perhaps you forgot a comma? (945164507.py, line 2)

## Exercise 2 
In order to find information in the web we can use microformats such as [hRecipe](https://microformats.org/wiki/hrecipe) or Schema.org's [Recipe](https://schema.org/Recipe). But first, we'll show you how to find arbitrary tags in a webpage.


### Task 2 
Parsing data for a <sub><sup>veggie</sup></sub> spaghetti alla carbonara recipe (from Example 2-7 in Mining the Social Web).

In [7]:
import requests
import json
from bs4 import BeautifulSoup

# A yummy webpage (feel free to change to your likings.)
URL = "https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara/"

# requests will return the html found at the given webpage...
page = requests.get(URL)
# ...and a BeautifulSoup object can be created from its content.
soup = BeautifulSoup(page.content, 'html.parser')

listchildren = list(soup.children)
# print(listchildren)

We can find any element in the page through *css tag selectors*
You can find them all [here](https://www.w3schools.com/cssref/css_selectors.asp), but shortly these are "." for classes, # for ids and plain text for the element name.


You can also combine them, so that looking for ".class1.class2" would select all elements displaying both classes. For a deeper overview please check the above link (or google "html tag selectors"). 

In [8]:
print(len(listchildren)) # we can see here how many children the html doc has got.
ingredients_unparsed = soup.select_one(".tasty-recipes-ingredients")
# let's get all the "list item" elements in a list:
ing_unp = ingredients_unparsed.findAll('li')
print(ing_unp)

2
[<li><span data-amount="1">1</span> pound spaghetti noodles</li>, <li><span data-amount="0.5" data-unit="cup">½ cup</span> smoked mozzarella cheese</li>, <li><span data-amount="0.5" data-unit="cup">½ cup</span> grated Parmesan cheese, plus more for serving</li>, <li><span data-amount="4">4</span> egg yolks</li>, <li><span data-amount="1" data-unit="cup">1 cup</span> frozen Earthbound Farm Organic peas</li>, <li><span data-amount="8" data-unit="cup">8 cups</span> Earthbound Farm Organic spinach</li>, <li><span data-amount="3" data-unit="tablespoon">3 tablespoons</span> butter</li>, <li><a class="tasty-link" data-tasty-links-no-disclosure="" href="https://www.acouplecooks.com/what-is-kosher-salt/" rel="noopener" target="_blank">Kosher salt</a></li>, <li>Fresh ground black pepper</li>]


Mmmh... not so pretty yet. How about listing their items using the text method?

In [9]:

ingredients = [t.text for t in ing_unp]
print("Ingredients:\n")
# [print(i) for i in ingredients]  # Also prints the generator
# Instead
for ing in ingredients:
    print(ing)

Ingredients:

1 pound spaghetti noodles
½ cup smoked mozzarella cheese
½ cup grated Parmesan cheese, plus more for serving
4 egg yolks
1 cup frozen Earthbound Farm Organic peas
8 cups Earthbound Farm Organic spinach
3 tablespoons butter
Kosher salt
Fresh ground black pepper


Good. Now the instructions:

In [10]:
instructions_unparsed = soup.select_one(".tasty-recipes-instructions")
instructions_unparsed = instructions_unparsed.findAll("li")
print(instructions_unparsed)

[<li id="instruction-step-1">In a large pot, combine 6 quarts of water with 2 tablespoons <a class="tasty-link" data-tasty-links-no-disclosure="" href="https://www.acouplecooks.com/what-is-kosher-salt/" rel="noopener" target="_blank">kosher salt</a> and bring it to a boil.</li>, <li id="instruction-step-2">Grate the Parmesan and mozzarella cheese. Carefully separate four egg yolks and set aside.</li>, <li id="instruction-step-3">Once boiling, add the pasta and cook until the pasta is just about al dente, about 7 minutes; then add peas and spinach and cook for 1 minute. Reserve 1 cup cooking water, and then drain the pasta and vegetables.</li>, <li id="instruction-step-4">In a skillet, melt the butter, then stir in the cheeses, ¼ cup pasta water, and ¼ teaspoon <a class="tasty-link" data-tasty-links-no-disclosure="" href="https://www.acouplecooks.com/what-is-kosher-salt/" rel="noopener" target="_blank">kosher salt</a>. Stir in the pasta and vegetables until creamy over low heat, adding 

Let's finish off with the title:

In [11]:
title_unparsed = soup.select_one(".post-header") # 
categorical_title = title_unparsed.text.split("›") # website specific divider.
recipe_title = categorical_title[-1].strip() # let's remove that ugly space at the beginning.
recipe_title

'Vegetarian Carbonara vegetarianJump to Recipeby Sonja OverhiserBuy Our Cookbook'

## Task 2.1
Now it's your turn. Create a function that can scrape any recipe webpage from the same website (other websites will have different class tags). 

Make sure to:

- return itemized content (e.g. ingredients) in a list. You may want to use a list comprehension here.
- Not all items have been cleaned of their html markdown (see variables ```ingredients``` vs. ```instructions_unparsed```. Make sure to return a list with human readable content (i.e. by using the ```.text``` attribute).


In [22]:
# -*- coding: utf-8 -*-

import requests
import json
from bs4 import BeautifulSoup

# Pass in a URL containing hRecipe, such as
# https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/

URL = "https://www.acouplecooks.com/best-jackfruit-tacos/"#YOUR RECIPE HERE/

# Parse out some of the pertinent information for a recipe.
# See http://microformats.org/wiki/hrecipe.

def parse_website(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # You code here
    # Parse header and get the title
    title_unparsed = soup.select_one(".post-header") # 
    categorical_title = title_unparsed.text.split("›") # website specific divider.
    recipe_title = categorical_title[-1].strip() # let's remove that ugly space at the beginning.
    fn = recipe_title

    # Ingredients
    ingredients_unparsed = soup.select_one(".tasty-recipes-ingredients")
    # let's get all the "list item" elements in a list:
    ing_unp = ingredients_unparsed.findAll('li')
    ingredients = [t.text for t in ing_unp]

    # Instructions
    instructions_unparsed = soup.select_one(".tasty-recipes-instructions")
    instructions_unparsed = instructions_unparsed.findAll("li")
    instructions = [t.text for t in instructions_unparsed]
    for ing in ingredients:
        print(ing)
    return instructions
    
recipe = parse_website(URL)
print("recipe:")
print (recipe)

2 15-ounce cans green jackfruit in water or brine
½ cup minced yellow onion
3 cloves garlic
2 tablespoons extra virgin olive oil
1 to 1 1/2 tablespoons adobo sauce (from 1 can chipotle peppers in adobo)
2 tablespoons cocoa powder
2 tablespoons tomato paste
2 teaspoons ground cumin
1 teaspoon oregano
¼ teaspoon kosher salt
½ cup water
8 tortillas (or more small street-style tortillas)
Chopped romaine
Fresh cilantro
Salsa Fresca or any fresh salsa
Refried beans, to serve on the side* (canned, Homemade Refried Beans, Instant Pot refried beans or Refried Black Beans)
recipe:
['Rinse and drain the jackfruit in a colander, pressing down to extract as much water as possible. Run your hands through the pieces, pulling and separating them into shreds with your fingers.', 'Mince the onion. Mince the garlic.', 'Heat the olive oil in a large skillet over medium heat. Add onion and garlic and saute for 3 to 4 minutes until tender and fragrant, but before the garlic browns. Add the jackfruit and rem

But How can we get information not only from one website,  but from all? 

The answer: microformats.

But rather than extracting with information manually from the schema.org or hRecipe microformats, we can use a package, ```scrape-schema-recipe``` 

Feel free to experiment with it. 

### Task 2.2
hRecipe is a microformat specifically created for recipes.
Can you for example easily compare different dessert recipe ingredients? For inspiration you can look back at the exercises you did in Hands-on session 1 where you compared different sets of tweets.

In [59]:
def fun(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # You code here
    # Parse header and get the title
    title_unparsed = soup.select_one(".post-header") # 
    categorical_title = title_unparsed.text.split("›") # website specific divider.
    recipe_title = categorical_title[-1].strip() # let's remove that ugly space at the beginning.
    fn = recipe_title

    # Ingredients
    ingredients_unparsed = soup.select_one(".tasty-recipes-ingredients")
    # let's get all the "list item" elements in a list:
    ing_unp = ingredients_unparsed.findAll('li')
    ingredients = [t.text for t in ing_unp]
    return fn, ingredients

title1, ing1 = fun("https://www.acouplecooks.com/apple-dumplings/")
title2, ing2 =fun("https://www.acouplecooks.com/pumpkin-ice-cream/")

print("both recipes contain:")
for _ in ing1:
    for __ in ing2:
        if _.split(" ", 1)[1:] == __.split(" ", 1)[1:]:
            print(_.split(" ", 1)[1])

print()
print(title1)
print(ing1)

print(title2)
print(ing2)

both recipes contain:
teaspoon kosher salt
teaspoon allspice
cup brown sugar

Apple Dumplings (Grandma’s Recipe) vegan vegetarianJump to Recipeby Sonja OverhiserBuy Our Cookbook
['2 cups all-purpose flour', '1 teaspoon kosher salt', '¼ teaspoon baking powder', '¾ cup unsalted butter*', '½ cup milk', '6 medium crisp tart apples\xa0', '6 tablespoons granulated sugar\xa0', '1 teaspoon cinnamon', '½ teaspoon allspice', '¼ teaspoon ginger', '3 tablespoons unsalted butter', '2/3 cup brown sugar', '⅓ cup water', '¼ teaspoon cinnamon', '2 tablespoons unsalted butter']
Pumpkin Ice Cream vegetarianJump to Recipeby Sonja OverhiserBuy Our Cookbook
['2 tablespoons cornstarch', '1 cup whole milk', '½ cup granulated sugar', '½ cup brown sugar', '2 teaspoons cinnamon', '1 teaspoon ground ginger', '½ teaspoon allspice', '¼ teaspoon nutmeg', '1 ½ cups heavy cream', '1 cup pumpkin puree', '1 tablespoon vanilla extract', '⅛ teaspoon kosher salt']


## Exercise 3

Schema.org is one of the most widely used annotations formats. Schema.org is a multipurpose  template that has been created by a consortium consisting of Yahoo!, Google and Microsoft. It can describe entities, events, products etc. Check out the vocabulary specs on Schema.org.

### Task 3

Parsing schema.org microdata. To parse this data you need to install the rdflib-microdata package, which you have done in one of the previous steps.



In [48]:
from rdflib import Graph

# Source: https://www.youtube.com/watch?v=sCU214rbRZ0
# Pass in a URL containing Schema.org microformats
URL = "http://dbpedia.org/resource/Micheal_Jackson"

# Initialize a graph
g = Graph()

# Parse in an RDF file graph dbpedia
result = g.parse(location=URL)

# Loop through first 10 triples in the graph
for index, (sub, pred, obj) in enumerate(g):
    print(sub, pred, obj)
    if index == 10:
        break

http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageWikiLink http://dbpedia.org/resource/Michael_Jackson
http://dbpedia.org/resource/Micheal_Jackson http://xmlns.com/foaf/0.1/isPrimaryTopicOf http://en.wikipedia.org/wiki/Micheal_Jackson
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageRevisionID 1056738079
http://dbpedia.org/resource/Micheal_Jackson http://www.w3.org/2000/01/rdf-schema#label Micheal Jackson
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/property/wikiPageUsesTemplate http://dbpedia.org/resource/Template:R_from_misspelling
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageLength 68
http://dbpedia.org/resource/Micheal_Jackson http://dbpedia.org/ontology/wikiPageID 14995602
http://dbpedia.org/resource/Micheal_Jackson http://www.w3.org/ns/prov#wasDerivedFrom http://en.wikipedia.org/wiki/Micheal_Jackson?oldid=1056738079&ns=0
http://dbpedia.org/resource/Micheal_Jackson htt

In [49]:
# Print the size of the Graph
print(f'Graph has {len(g)} facts')

Graph has 9 facts


In [50]:
# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format='ttl'))

@prefix ns1: <http://dbpedia.org/ontology/> .
@prefix ns2: <http://xmlns.com/foaf/0.1/> .
@prefix ns3: <http://dbpedia.org/property/> .
@prefix ns4: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://dbpedia.org/resource/Micheal_Jackson> rdfs:label "Micheal Jackson"@en ;
    ns1:wikiPageID 14995602 ;
    ns1:wikiPageLength "68"^^xsd:nonNegativeInteger ;
    ns1:wikiPageRedirects <http://dbpedia.org/resource/Michael_Jackson> ;
    ns1:wikiPageRevisionID 1056738079 ;
    ns1:wikiPageWikiLink <http://dbpedia.org/resource/Michael_Jackson> ;
    ns3:wikiPageUsesTemplate <http://dbpedia.org/resource/Template:R_from_misspelling> ;
    ns4:wasDerivedFrom <http://en.wikipedia.org/wiki/Micheal_Jackson?oldid=1056738079&ns=0> ;
    ns2:isPrimaryTopicOf <http://en.wikipedia.org/wiki/Micheal_Jackson> .




### Task 3.1 
Compare the schema.org information about a band on last.fm to the Facebook Open Graph information about the same band from Facebook. What are the differences? Which format do you think supports better interoperability?

In [54]:
# Initialize a graph
g = Graph()
URL_LAST = "https://www.last.fm/music/Pink+Floyd"
URL_DB = "http://dbpedia.org/resource/Pink_Floyd"

# Parse in an RDF file graph dbpedia
result = g.parse(location=URL_DB)

# Loop through first 10 triples in the graph
for index, (sub, pred, obj) in enumerate(g):
    print(sub, pred, obj)
    if index == 10:
        break

http://dbpedia.org/resource/Pink_Floyd http://dbpedia.org/ontology/wikiPageExternalLink https://books.google.com/books%3Fid=nQ1f7Vasrv8C
http://dbpedia.org/resource/Pink_Floyd http://www.w3.org/2000/01/rdf-schema#label Pink Floyd
http://dbpedia.org/resource/Pink_Floyd http://dbpedia.org/ontology/wikiPageWikiLink http://dbpedia.org/resource/The_Guardian
http://dbpedia.org/resource/Pink_Floyd http://dbpedia.org/ontology/wikiPageWikiLink http://dbpedia.org/resource/Top_of_the_Pops
http://dbpedia.org/resource/Pink_Floyd http://dbpedia.org/ontology/abstract Pink Floyd [pɪŋk flɔɪd] est un groupe britannique de rock originaire de Londres. Le groupe débute avec un premier album de musique psychédélique pour ensuite bifurquer vers le rock progressif. Formé en 1965, il est considéré comme un pionnier et représentant majeur de ces styles musicaux. Il est reconnu pour sa musique planante et expérimentale, ses textes philosophiques et satiriques, ses albums-concept et ses performances en concert or

In [81]:
page = requests.get(URL_LAST)
soup = BeautifulSoup(page.content, 'html.parser')
data = soup.select_one(".main-content")
print(soup.select_one("div", {"itemtype": "http://schema.org"}))

<div data-require="tracking/tealium-utag-set" data-tealium-data='{"siteSection": "music", "pageType": "artist_door", "pageName": "music/artist/overview", "nativeEventTracking": true, "userState": "not authenticated", "userType": "anon", "musicArtistName": "Pink Floyd", "artist": "pinkfloyd", "ar": "pinkfloyd,davidgilmour,rogerwaters,sydbarrett,ledzeppelin", "tag": "progressiverock,psychedelicrock,classicrock,rock,psychedelic"}' data-tealium-environment="prod" id="initial-tealium-data"></div>


We can see that dbpedia gives a more extensive result, that can be better interpreted, but for programming the one from last fm is easier to manipulate

### Task 3.2
Explore the various microformats at http://microformats.org/ and compare the output of the exercises with the output of http://microformats.org/. Think about possible microformats you want to support in your final assignment and read up on how to parse them.

https://github.com/microformats/mf2py

We will use a Json basesd microformat because they are universal and easy to manipulate in python. Storage of json object is also not a problem with non SQL databases